In [12]:
import subprocess
subprocess.run('wget https://raw.githubusercontent.com/choi-hyunsoo/NLP2023/main/download.py', shell=True)
subprocess.run('wget https://raw.githubusercontent.com/choi-hyunsoo/NLP2023/main/get-surnames-data.sh', shell=True)
subprocess.run('chmod +x get-surnames-data.sh', shell=True)
subprocess.run('./get-surnames-data.sh', shell=True)

CompletedProcess(args='./get-surnames-data.sh', returncode=1)

In [1]:
!wget https://raw.githubusercontent.com/choi-hyunsoo/NLP2023/main/download.py
!wget https://raw.githubusercontent.com/choi-hyunsoo/NLP2023/main/get-surnames-data.sh

!chmod +x get-surnames-data.sh
!./get-surnames-data.sh

--2024-03-30 14:39:41--  https://raw.githubusercontent.com/choi-hyunsoo/NLP2023/main/download.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1568 (1.5K) [text/plain]
Saving to: 'download.py.3'

     0K .                                                     100% 8.50M=0s

2024-03-30 14:39:41 (8.50 MB/s) - 'download.py.3' saved [1568/1568]

--2024-03-30 14:39:41--  https://raw.githubusercontent.com/choi-hyunsoo/NLP2023/main/get-surnames-data.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 508 [text/plain]
Saving to: 'get-surna

In [5]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [7]:
args = Namespace(
    raw_dataset_csv = "./surnames/surnames.csv",  # dataset
    train_proportion = 0.7, # train data
    val_proportion = 0.15, # valid data
    test_proportion = 0.15, # test data
    output_munged_csv = "./surnames/surnames_with_splits.csv", # output data
    seed = 1337
    )

In [8]:
surnames = pd.read_csv(args.raw_dataset_csv, header=0)

In [9]:
surnames.head()

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


In [10]:
set(surnames.nationality)

{'Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Greek',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Polish',
 'Portuguese',
 'Russian',
 'Scottish',
 'Spanish',
 'Vietnamese'}

In [11]:
surnames.info

<bound method DataFrame.info of         surname nationality
0      Woodford     English
1          Coté      French
2          Kore     English
3         Koury      Arabic
4        Lebzak     Russian
...         ...         ...
10975  Quraishi      Arabic
10976   Innalls     English
10977      Król      Polish
10978    Purvis     English
10979  Messerli      German

[10980 rows x 2 columns]>

In [13]:
surnames.describe

<bound method NDFrame.describe of         surname nationality
0      Woodford     English
1          Coté      French
2          Kore     English
3         Koury      Arabic
4        Lebzak     Russian
...         ...         ...
10975  Quraishi      Arabic
10976   Innalls     English
10977      Król      Polish
10978    Purvis     English
10979  Messerli      German

[10980 rows x 2 columns]>

In [15]:
set(surnames.nationality)

{'Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Greek',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Polish',
 'Portuguese',
 'Russian',
 'Scottish',
 'Spanish',
 'Vietnamese'}

In [16]:
by_nationality = collections.defaultdict(list)
for _, row in surnames.iterrows():
    by_nationality[row.nationality].append(row.to_dict())
    # dictionary 형태로 저장

In [17]:
by_nationality

defaultdict(list,
            {'English': [{'surname': 'Woodford', 'nationality': 'English'},
              {'surname': 'Kore', 'nationality': 'English'},
              {'surname': 'Essop', 'nationality': 'English'},
              {'surname': 'Jefferson', 'nationality': 'English'},
              {'surname': 'Dorrington', 'nationality': 'English'},
              {'surname': 'Jeffries', 'nationality': 'English'},
              {'surname': 'Douthwaite', 'nationality': 'English'},
              {'surname': 'Readle', 'nationality': 'English'},
              {'surname': 'Jones', 'nationality': 'English'},
              {'surname': 'Topham', 'nationality': 'English'},
              {'surname': 'Bellamy', 'nationality': 'English'},
              {'surname': 'Leggett', 'nationality': 'English'},
              {'surname': 'Lilley', 'nationality': 'English'},
              {'surname': 'Ayliffe', 'nationality': 'English'},
              {'surname': 'Twiggs', 'nationality': 'English'},
            

In [19]:
by_nationality.items()

dict_items([('English', [{'surname': 'Woodford', 'nationality': 'English'}, {'surname': 'Kore', 'nationality': 'English'}, {'surname': 'Essop', 'nationality': 'English'}, {'surname': 'Jefferson', 'nationality': 'English'}, {'surname': 'Dorrington', 'nationality': 'English'}, {'surname': 'Jeffries', 'nationality': 'English'}, {'surname': 'Douthwaite', 'nationality': 'English'}, {'surname': 'Readle', 'nationality': 'English'}, {'surname': 'Jones', 'nationality': 'English'}, {'surname': 'Topham', 'nationality': 'English'}, {'surname': 'Bellamy', 'nationality': 'English'}, {'surname': 'Leggett', 'nationality': 'English'}, {'surname': 'Lilley', 'nationality': 'English'}, {'surname': 'Ayliffe', 'nationality': 'English'}, {'surname': 'Twiggs', 'nationality': 'English'}, {'surname': 'Peebles', 'nationality': 'English'}, {'surname': 'Clowes', 'nationality': 'English'}, {'surname': 'Winship', 'nationality': 'English'}, {'surname': 'Ingleby', 'nationality': 'English'}, {'surname': 'Dundon', 'nati

In [20]:
final_list = []
np.random.seed(args.seed)
for _, item_list in sorted(by_nationality.items()):
    np.random.shuffle(item_list)
    n = len(item_list)
    
    n_train = int(args.train_proportion*n)
    n_val = int(args.val_proportion*n)
    n_test = int(args.test_proportion*n)

    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:]:
        item['split'] = 'test'

    final_list.extend(item_list)

In [21]:
final_surnames = pd.DataFrame(final_list)

In [23]:
final_surnames.head()

,surname,nationality,split
0,Totah,Arabic,train
1,Abboud,Arabic,train
2,Fakhoury,Arabic,train
3,Srour,Arabic,train
4,Sayegh,Arabic,train


In [24]:
final_surnames.split.value_counts()

split
train    7680
test     1660
val      1640
Name: count, dtype: int64

In [25]:
final_surnames.to_csv(args.output_munged_csv, index=False) # index가 csv에 저장되지 않음